In [4]:
import tensorflow as tf
import numpy as np
from tensorflow.contrib.layers.python.layers.layers import convolution
from tensorflow.python.ops.nn_impl import relu_layer
import sys
sys.path.append('../scripts')
import dataImporter
from random import shuffle

In [6]:
epoch_count = 10
batch_size = 32
neurons_fc = 32
conv_filter_size = 5
feature_maps_layer1 = 16
feature_maps_layer2 = 32
output_vector_size = 4
learn_rate = 1e-4
eval_data_amount = 600

gesture_data_dir = '../../gestureData' 

#Variables
#source http://cs231n.github.io/neural-networks-2/
bias_init_factor = 0.1
#random_initializer = tf.contrib.layers.xavier_initializer_conv2d(uniform=True, seed=None, dtype=tf.float32
random_initializer = tf.truncated_normal_initializer(stddev=0.1, dtype=tf.float32)
#random_initializer = tf.zeros_initializer

In [7]:
def conv3d(x, W):
    return tf.nn.conv3d(x, W, strides=[1, 1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool3d(x, ksize=[1, 2, 2, 2, 1], strides=[1, 2, 2, 2, 1], padding='SAME')

with tf.name_scope('input'):
    input_data = tf.placeholder(tf.float32, shape=[None,40,40,40], name='input_data')
    

# First "Layer Stack"
with tf.name_scope('conv_layer_1'):
    filters_layer1 = tf.get_variable("filter1_variables",
                                     [conv_filter_size, 
                                      conv_filter_size, conv_filter_size,
                                      1,
                                      feature_maps_layer1],
                                     initializer = random_initializer)
    bias_layer1 = tf.get_variable("bias1_variables",
                                  [feature_maps_layer1],
                                  initializer = tf.constant_initializer(bias_init_factor))
    reshapted_input_data = tf.reshape(input_data, [-1, 40, 40, 40, 1])
    convolution_layer1 = conv3d(reshapted_input_data, filters_layer1) + bias_layer1

with tf.name_scope('pooling_layer_1'):
    pooling_layer1 = max_pool_2x2(convolution_layer1)


with tf.name_scope('ReLU_layer_1'):
    relu_layer1 = tf.nn.relu(pooling_layer1)
    

    
# Second "Layer Stack"
with tf.name_scope('conv_layer_2'):
    filters_layer2 = tf.get_variable("filter2_variables",
                                     [conv_filter_size,
                                      conv_filter_size,
                                      conv_filter_size,
                                      feature_maps_layer1,
                                      feature_maps_layer2],
                                     initializer = random_initializer)
    bias_layer2 = tf.get_variable("bias2_variables",
                                  [feature_maps_layer2],
                                  initializer = tf.constant_initializer(bias_init_factor))    
    convolution_layer2 = conv3d(relu_layer1, filters_layer2) + bias_layer2
    
with tf.name_scope('pooling_layer_2'):
    pooling_layer2 = max_pool_2x2(convolution_layer2)

with tf.name_scope('ReLU_layer_2'):
    relu_layer2 = tf.nn.relu(pooling_layer2)

    
# Start Fully connected layers    
with tf.name_scope('fc_layer_1'):
    weights_fc1 = tf.get_variable("fc1_variables",
                            [10 * 10 * 10 * feature_maps_layer2, neurons_fc],
                            initializer = random_initializer)
    bias_fc1 = tf.get_variable("bias_fc1_variables",
                               [neurons_fc],
                               initializer = tf.constant_initializer(bias_init_factor))

    pool_layer2_flat = tf.reshape(relu_layer2, [-1, 10 * 10 * 10*feature_maps_layer2])
    fc1 = tf.nn.relu(tf.matmul(pool_layer2_flat, weights_fc1) + bias_fc1)
    
with tf.name_scope('dropout_layer'):
    keep_prob = tf.placeholder(tf.float32)
    fc1_dropout = tf.nn.dropout(fc1, keep_prob)

with tf.name_scope('fc_layer_2'):
    weights_fc2 = tf.get_variable("fc2_variables",
                                  [neurons_fc, output_vector_size],
                                  initializer = random_initializer)
    bias_fc2 = tf.get_variable("bias_fc2_variables",
                               [output_vector_size],
                               initializer = tf.constant_initializer(bias_init_factor))
    
    output_class_probabilities = tf.matmul(fc1_dropout, weights_fc2) + bias_fc2

In [8]:
desired_label = tf.placeholder(tf.float32, shape=[None,4], name='desired_label')

with tf.name_scope('error'):
    error = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=desired_label, logits=output_class_probabilities))

with tf.name_scope('optimizer'):
    optimizer = tf.train.AdamOptimizer(learn_rate)
    train_step = optimizer.minimize(error)
        
with tf.name_scope('accuracy'):
    correct_prediction = tf.equal(tf.argmax(output_class_probabilities, 1), tf.argmax(desired_label, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.



In [11]:
train_data = []

In [12]:
train_data = train_data + dataImporter.get_discrete3D_data(dataImporter.read_data_from_file(gesture_data_dir + "/circle/trainData.json"), True, True)
train_data = train_data + dataImporter.get_discrete3D_data(dataImporter.read_data_from_file(gesture_data_dir + "/v/trainData.json"), True, True)
train_data = train_data + dataImporter.get_discrete3D_data(dataImporter.read_data_from_file(gesture_data_dir + "/circle/trainData2.json"), True, True)
train_data = train_data + dataImporter.get_discrete3D_data(dataImporter.read_data_from_file(gesture_data_dir + "/line/trainData.json"), True, True)
train_data = train_data + dataImporter.get_discrete3D_data(dataImporter.read_data_from_file(gesture_data_dir + "/line/trainData2.json"), True, True)
train_data = train_data + dataImporter.get_discrete3D_data(dataImporter.read_data_from_file(gesture_data_dir + "/v/trainData2.json"), True, True)
train_data = train_data + dataImporter.get_discrete3D_data(dataImporter.read_data_from_file(gesture_data_dir + "/wave/trainData.json"), True, True)
train_data = train_data + dataImporter.get_discrete3D_data(dataImporter.read_data_from_file(gesture_data_dir + "/wave/trainData2.json"), True, True)
shuffle(train_data)

imported data of 50 gestures
starting to cleanup gesture Data...
done cleaning up. Removed 2 gestures
starting to close gaps in gesture coordinate data...
done closing gaps... Added 1001 coordinates
augumented input data 384
starting to transpose 384 discrete gestures
having 768 gestures now
starting to flip 768 discrete gestures on axis 0
having 1536 gestures now
starting to flip 1536 discrete gestures on axis 1
having 3072 gestures now
starting to flip 3072 discrete gestures on axis 2
having 6144 gestures now
starting to shift 6144 gestures
having 12288 now
starting to shift down 12288 gestures
having 24576 now
starting to shift back 24576 gestures
having 49152 now
 
compressing...
imported data of 50 gestures
starting to cleanup gesture Data...
done cleaning up. Removed 0 gestures
starting to close gaps in gesture coordinate data...
done closing gaps... Added 188 coordinates
augumented input data 400
starting to transpose 400 discrete gestures
having 800 gestures now
starting to fli

In [13]:
eval_data = []
#ensure random eval data is equally distributed through all gesture types!
eval_data_class_counts = {0:0,1:0,2:0,3:0}
while len(eval_data) < eval_data_amount:
    element = train_data.pop()
    element_class_id = np.array(element[1]).nonzero()[0][0]
    if eval_data_class_counts[element_class_id]<int(eval_data_amount/4):
        eval_data_class_counts[element_class_id] = eval_data_class_counts[element_class_id] + 1
        eval_data.append(element)
    else:
        train_data.insert(0,element)
    
decompressed_batch = dataImporter.decompress_3D(eval_data)
eval_input_batch = []
eval_label_batch = []
for j in range(eval_data_amount):
    eval_input_batch.append(decompressed_batch[j][0])
    eval_label_batch.append(decompressed_batch[j][1])

In [14]:
test_data = []
test_data = dataImporter.get_discrete3D_data(dataImporter.read_data_from_file(gesture_data_dir + "/line/testData.json"), True, False)
test_data = test_data + dataImporter.get_discrete3D_data(dataImporter.read_data_from_file(gesture_data_dir + "/circle/testData.json"), True, False)
test_data = test_data + dataImporter.get_discrete3D_data(dataImporter.read_data_from_file(gesture_data_dir + "/v/testData.json"), True, False)
test_data = test_data + dataImporter.get_discrete3D_data(dataImporter.read_data_from_file(gesture_data_dir + "/wave/testData.json"), True, False)
test_data_count = len(test_data)
shuffle(test_data)
print("its " + repr(test_data_count) + " testdata")

decompressed_batch = dataImporter.decompress_3D(test_data)
test_input_batch = []
test_label_batch = []
for j in range(test_data_count):
    test_input_batch.append(decompressed_batch[j][0])
    test_label_batch.append(decompressed_batch[j][1])
    

imported data of 50 gestures
starting to cleanup gesture Data...
done cleaning up. Removed 1 gestures
starting to close gaps in gesture coordinate data...
done closing gaps... Added 193 coordinates
 
compressing...
imported data of 50 gestures
starting to cleanup gesture Data...
done cleaning up. Removed 0 gestures
starting to close gaps in gesture coordinate data...
done closing gaps... Added 875 coordinates
 
compressing...
imported data of 50 gestures
starting to cleanup gesture Data...
done cleaning up. Removed 0 gestures
starting to close gaps in gesture coordinate data...
done closing gaps... Added 370 coordinates
 
compressing...
imported data of 50 gestures
starting to cleanup gesture Data...
done cleaning up. Removed 0 gestures
starting to close gaps in gesture coordinate data...
done closing gaps... Added 875 coordinates
 
compressing...
its 199 testdata


In [15]:
fremdgesten_data = []
fremdgesten_data = dataImporter.get_discrete3D_data(dataImporter.read_data_from_file(gesture_data_dir + "/fremdGestendaten/person1.json"), True, False)
fremdgesten_data = fremdgesten_data + dataImporter.get_discrete3D_data(dataImporter.read_data_from_file(gesture_data_dir + "/fremdGestendaten/person2.json"), True, False)
fremdgesten_data = fremdgesten_data + dataImporter.get_discrete3D_data(dataImporter.read_data_from_file(gesture_data_dir + "/fremdGestendaten/person3.json"), True, False)
fremdgesten_data = fremdgesten_data + dataImporter.get_discrete3D_data(dataImporter.read_data_from_file(gesture_data_dir + "/fremdGestendaten/person4.json"), True, False)
fremdgesten_data = fremdgesten_data + dataImporter.get_discrete3D_data(dataImporter.read_data_from_file(gesture_data_dir + "/fremdGestendaten/person5.json"), True, False)
fremdgesten_data_count = len(fremdgesten_data)
shuffle(fremdgesten_data)
print("its " + repr(fremdgesten_data_count) + " fremdgesten")

decompressed_batch = dataImporter.decompress_3D(fremdgesten_data)
fremdgesten_input_batch = []
fremdgesten_label_batch = []
for j in range(fremdgesten_data_count):
    fremdgesten_input_batch.append(decompressed_batch[j][0])
    fremdgesten_label_batch.append(decompressed_batch[j][1])





imported data of 43 gestures
starting to cleanup gesture Data...
done cleaning up. Removed 1 gestures
starting to close gaps in gesture coordinate data...
done closing gaps... Added 628 coordinates
 
compressing...
imported data of 37 gestures
starting to cleanup gesture Data...
done cleaning up. Removed 1 gestures
starting to close gaps in gesture coordinate data...
done closing gaps... Added 1237 coordinates
 
compressing...
imported data of 36 gestures
starting to cleanup gesture Data...
done cleaning up. Removed 3 gestures
starting to close gaps in gesture coordinate data...
done closing gaps... Added 1548 coordinates
 
compressing...
imported data of 43 gestures
starting to cleanup gesture Data...
done cleaning up. Removed 1 gestures
starting to close gaps in gesture coordinate data...
done closing gaps... Added 1287 coordinates
 
compressing...
imported data of 36 gestures
starting to cleanup gesture Data...
done cleaning up. Removed 4 gestures
starting to close gaps in gesture c

In [16]:
writer = tf.summary.FileWriter('../../tensorboard/tmp', graph=tf.get_default_graph())

acuracy_op_eval = tf.summary.scalar("accuracy_eval", accuracy)
acuracy_op_test = tf.summary.scalar("accuracy_test", accuracy)
acuracy_op_train = tf.summary.scalar("accuracy_train", accuracy)
error_op_eval = tf.summary.scalar("error_eval", error)
error_op_test = tf.summary.scalar("error_test", error)
error_op_train = tf.summary.scalar("error_train", error)

In [17]:
with tf.name_scope('session'):
    session = tf.Session()
    init = tf.global_variables_initializer()
    session.run(init)

In [20]:
#restore if there is already a trained model..
saver = tf.train.Saver()
saver.restore(session, "./storedSessions/10Each40Epoch/model10Each.ckpt")

INFO:tensorflow:Restoring parameters from ./storedSessions/10Each40Epoch/model10Each.ckpt


InvalidArgumentError: Unsuccessful TensorSliceReader constructor: Failed to get matching files on ./storedSessions/10Each40Epoch/model10Each.ckpt: Not found: ./storedSessions/10Each40Epoch; No such file or directory
	 [[Node: save_1/RestoreV2 = RestoreV2[dtypes=[DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, ..., DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT], _device="/job:localhost/replica:0/task:0/device:CPU:0"](_arg_save_1/Const_0_0, save_1/RestoreV2/tensor_names, save_1/RestoreV2/shape_and_slices)]]

Caused by op 'save_1/RestoreV2', defined at:
  File "/usr/local/Cellar/python/3.6.5/Frameworks/Python.framework/Versions/3.6/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/local/Cellar/python/3.6.5/Frameworks/Python.framework/Versions/3.6/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/Users/tobirohrer/evonik/tf/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.la

In [19]:
iterations = int(len(train_data)/batch_size)
print("training on " + repr(len(train_data)) + " gestures")
print("evaluating on " + repr(len(eval_data)) + " gestures")
print("thats " + repr(iterations) + " iterations per epoch")
print("...and its " + repr(epoch_count) + " epochs, so lean back and wait!")

with tf.name_scope('training...'):
    for e in range(epoch_count):
        print("")
        print("starting epoch " + repr(e))
        shuffle(train_data)
        for i in range(iterations):
            
            compressed_batch = []
            for j in range(batch_size):
                compressed_batch.append(train_data[i*batch_size+j])
            decompressed_batch = dataImporter.decompress_3D(compressed_batch)
            input_batch = []
            label_batch = []
            for j in range(batch_size):
                input_batch.append(decompressed_batch[j][0])
                label_batch.append(decompressed_batch[j][1])

            session.run(train_step,feed_dict={input_data: input_batch, desired_label: label_batch, keep_prob: 0.5})
            
            #monitor training process:
            if(i%500 == 0):
                print(session.run(accuracy, feed_dict={input_data: eval_input_batch, desired_label: eval_label_batch, keep_prob: 1.0}))
            
            if(i%100 == 0):
                #tensorboard logging!
                train_data_batch = []
                for j in range(eval_data_amount):
                    train_data_batch.append(train_data[j])
                
                decompressed_train_batch = dataImporter.decompress_3D(train_data_batch)
                train_input_batch = []
                train_label_batch = []
                for j in range(eval_data_amount):
                    train_input_batch.append(decompressed_train_batch[j][0])
                    train_label_batch.append(decompressed_train_batch[j][1])

                
                error_summary_eval = session.run(error_op_eval,feed_dict={input_data: eval_input_batch, desired_label: eval_label_batch, keep_prob:1.0})
                error_summary_test = session.run(error_op_test,feed_dict={input_data: test_input_batch, desired_label: test_label_batch, keep_prob:1.0})
                error_summary_train = session.run(error_op_train,feed_dict={input_data: train_input_batch, desired_label: train_label_batch, keep_prob:1.0})
                
                acuracy_summary_eval = session.run(acuracy_op_eval,feed_dict={input_data: eval_input_batch, desired_label: eval_label_batch, keep_prob:1.0})
                acuracy_summary_test = session.run(acuracy_op_test,feed_dict={input_data: test_input_batch, desired_label: test_label_batch, keep_prob:1.0})
                acuracy_summary_train = session.run(acuracy_op_train,feed_dict={input_data: train_input_batch, desired_label: train_label_batch, keep_prob:1.0})
                
                writer.add_summary(error_summary_eval, i+((e)*iterations))
                writer.add_summary(acuracy_summary_eval, i+((e)*iterations))
                writer.add_summary(error_summary_test, i+((e)*iterations))
                writer.add_summary(acuracy_summary_test, i+((e)*iterations))
                writer.add_summary(error_summary_train, i+((e)*iterations))
                writer.add_summary(acuracy_summary_train, i+((e)*iterations))

training on 200104 gestures
evaluating on 600 gestures
thats 6253 iterations per epoch
...and its 10 epochs, so lean back and wait!

starting epoch 0
0.255


KeyboardInterrupt: 

In [12]:
writer.flush()
writer.close()



In [21]:
print("accuracy on testdata:")
print(session.run(accuracy,feed_dict={input_data: test_input_batch, desired_label: np.matrix(test_label_batch), keep_prob:1.0}))
print("accuracy on evaldata:")
print(session.run(accuracy,feed_dict={input_data: eval_input_batch, desired_label: np.matrix(eval_label_batch), keep_prob:1.0}))
print("accuracy on fremdgesten:")
print(session.run(accuracy,feed_dict={input_data: fremdgesten_input_batch, desired_label: np.matrix(fremdgesten_label_batch), keep_prob:1.0}))


accuracy on testdata:
accuracy on evaldata:
accuracy on fremdgesten:
0.25405404


# Visualisation (Draft)

In [22]:
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
xs,ys,zs = np.nonzero(eval_data[1][0])
ax.scatter(xs,zs,ys)
ax.set_xlabel('X Label')
ax.set_ylabel('Z Label')
ax.set_zlabel('Y Label')
plt.show()

conv_layer1 = session.run(convolution_layer1, {x: np.matrix(eval_data[1][0].flatten())})
transposed_conv_layer1 = tf.transpose(conv_layer1, [4, 1, 2, 3,0])

#its now an "array" of 3D cubes
for i in range(transposed_conv_layer1.shape[0]):
    single_feature_map = transposed_conv_layer1[i]
    print(i)
    #get rid of nonsense last dimension
    conv_3d = tf.reshape(single_feature_map, (40,40,40))
    #plot
    fig= plt.figure(i)
    ax = fig.add_subplot(111, projection='3d')
    x1, y1, z1 = np.nonzero(np.around(session.run(conv_3d)))
    ax.scatter(x1,z1,y1)
    ax.set_xlabel('X Label')
    ax.set_ylabel('Z Label')
    ax.set_zlabel('Y Label')
    plt.show()


ValueError: not enough values to unpack (expected 3, got 2)

In [25]:
transposed_filters_layer1 = tf.transpose(filters_layer1, [4, 0, 1, 2, 3])
for i in range(transposed_filters_layer1.shape[0]):
    single_filter = transposed_filters_layer1[i]
    print(i)
    #get rid of nonsense last dimension
    filter_3d = tf.reshape(single_filter, (conv_filter_size,conv_filter_size,conv_filter_size))
    #plot
    fig= plt.figure(i)
    ax = fig.add_subplot(111, projection='3d')
    x1, y1, z1 = np.nonzero(session.run(filter_3d))
    c = session.run(filter_3d).flatten()
    #dunkel ist mehr und nicht andersrum
    c = c*-1
    ax.scatter(x1, y1, z1, c=c, cmap=plt.gray())
    ax.set_xlabel('X Label')
    ax.set_ylabel('Z Label')
    ax.set_zlabel('Y Label')

    plt.show()

0


NameError: name 'plt' is not defined

In [23]:
%matplotlib
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt

transposed_filters_layer1 = tf.transpose(filters_layer1, [4, 0, 1, 2, 3])
single_filter = transposed_filters_layer1[0]
#get rid of nonsense last dimension
filter_3d = tf.reshape(single_filter, (conv_filter_size,conv_filter_size,conv_filter_size))
#plot
fig= plt.figure()
ax = fig.add_subplot(111, projection='3d')
x1, y1, z1 = np.nonzero(session.run(filter_3d))
c = session.run(filter_3d).flatten()
for i in range(len(c)):
    if(c[i] < 0.1):
        c[i] = 0
#dunkel ist mehr und nicht andersrum
c = c*-1
ax.scatter(x1, z1, y1, c=c, cmap=plt.gray())
ax.set_xlabel('X Label')
ax.set_ylabel('Z Label')
ax.set_zlabel('Y Label')
    
plt.show()

Using matplotlib backend: MacOSX
